# Machine Learning Pipelines

* Advantages of Machine Learning Pipelines
* Scikit-learn Pipeline
* Scikit-learn Feature Union
* Pipelines and Grid Search
* Case Study

**CASE STUDY:**
#### Corporate Messaging Case Study
This corporate message data is from one of the free datasets provided on the [Figure Eight Platform](https://www.figure-eight.com/data-for-everyone/), licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).